# Building Molecular Hamiltonian
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Overview

In this tutorial, we will demonstrate how to use Paddle Quantum's *qchem* module to build valid Hamiltonian for simulating chemical molecules on a quantum computer. We will go step by step how to build the second quantized Hamiltonian from a molecular structure and how to transform it to a set of Pauli matrices. 

Hamiltonian is a physical quantity related to the total energy of a physical system. In general, it can be represented as 

$$
\hat{H}=\hat{T}+\hat{V},\tag{1}
$$

where $\hat{T}$ is the kinetic energy and $\hat{V}$ is the potential energy. Hamiltonian is useful for various quantum algorithms, such as [variational quantum eigensolver](./VQE_EN.ipynb) and [Hamiltonian Simulation with Product Formula](./HamiltonianSimulation_EN.ipynb).

When trying to solve a chemistry problem with quantum mechanics, we also need to write down a Hamiltonian that describes the chemical system involved in the problem. Starting from this Hamiltonian, we can, in principle, calculate the ground state and excited states, and use the information to further explore all the physical properties of the quantum system. The dominant Hamiltonian of electronic problems has the form

$$
\hat{H}=\sum_{i=1}^N\left(-\frac{1}{2}\nabla_{x_i}^2\right)+\sum_{i=1}^N\sum_{j< i}\frac{1}{|x_i-x_j|}-\sum_{i=1}^N\sum_{I=1}^M\frac{Z_I}{|x_i-R_I|},\tag{2}
$$

when we use [atomic units](https://en.wikipedia.org/wiki/Hartree_atomic_units). Our electronic problem contains $N$ electrons and $M$ nucleus. We use $x_i$ to denote position of the $i$-th electron, and use $R_I$ to denote position of the $I$-th nuclei. 

This tutorial will have the following parts. Let's first talk about how to construct a molecule in *qchem*. After that, we will briefly describe how to calculate [Hartree Fock](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method) single particle orbitals by calling external quantum chemistry within Paddle Quantum. Next, we show how we can obtain molecular Hamiltonian using *qchem*'s `Molecule` class.

## Defining the molecular structure
In this example, we show how to construct water molecule from its chemical formula and coordinates of atoms. 

![h2o.png](figures/buildingmolecule-fig-h2o.png)

Within Paddle Quantum, we specify the atom as a list whose first element is the atomic symbol and the second element is another list that contains its Cartesian coordinate. The molecule is thus a bigger list composed of atoms' list.

**Note: As to the environment setting, please refer to [README.md](https://github.com/PaddlePaddle/Quantum/blob/master/README.md).**

In [1]:
# Eliminate noisy python warnings
import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
# in Angstrom
h2o_coords = [("H", [-0.02111417,0.8350417,1.47688078]),  # H stands for hydrogen element in water
                        ("O", [0.0, 0.0, 0.0]),                     # O stands for oxygen element in water
                        ("H", [-0.00201087,0.45191737,-0.27300254])]

## Calculate Hartree Fock orbitals
Hartree Fock method uses the [Slater determinant](https://en.wikipedia.org/wiki/Slater_determinant) to represent the $N$-electron wavefunction. It could provide us with a set of single particle orbitals which are often taken as input to more advanced quantum chemistry methods. 

Currently, Paddle Quantum uses PySCF [1] as its quantum chemistry engine, we will support more quantum chemistry toolkits, such as psi4, in the future (NOTE: PySCF currently only support Mac and Linux platform). We could use the `PySCFDriver` provided in `qchem` module to manage the quantum chemistry calculation and get the necessary information about the molecule. It takes molecular structure, total molecular charge, and spin multiplicity as its major inputs. We can also control the precision of Hartree Fock calculation by setting different [basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)) for `basis` keyword. 

In [4]:
from paddle_quantum.qchem import PySCFDriver

driver = PySCFDriver()
driver.load_molecule(
    atom=h2o_coords,         # Geometry of H2O molecule
    basis="sto-3g",          # Basis set for quantum chemistry calculation
    multiplicity=1,          # Spin multiplicity for molecule, since the total spin of H2O is S=0，its spin multiplicity is 2S+1=1
    charge=0,                 # Total charge of molecule, since H2O is charge neutral, its charge=0
    unit="Angstrom"
)
driver.run_scf()             # Perform Hartree Fock calculation

INFO:root:
#######################################
SCF Calculation (Classical)
#######################################
INFO:root:Basis: sto-3g


converged SCF energy = -73.9677038774737


INFO:root:SCF energy: -73.96770.


## Molecular Hamiltonian in second quantization form
When we study many electron quantum systems, it's often convenient to write Hamiltonian at the beginning of this tutorial in [second quantization](https://en.wikipedia.org/wiki/Second_quantization) representation 

$$
\hat{H}=\sum_{p,q}h_{pq}\hat{c}^{\dagger}_p\hat{c}_q+\frac{1}{2}\sum_{p,q,r,s}v_{pqrs}\hat{c}^{\dagger}_p\hat{c}^{\dagger}_q\hat{c}_r\hat{c}_s,\tag{3}$$

where $p$, $q$, $r$ and $s$ are Hartree Fock orbitals computed in the previous section. $\hat{c}^{\dagger}_p$ and $\hat{c}_q$ are creation and annihilation operations, respectively. The two coefficients $h_{pq}$ and $v_{pqrs}$ are called molecular integrals, and can be obtained from `PySCFDriver` in the following way.

In [5]:
import numpy as np 
np.set_printoptions(precision=4, linewidth=150)

hpq = driver.get_onebody_tensor("int1e_kin") + driver.get_onebody_tensor("int1e_nuc")
vpqrs = driver.get_twobody_tensor()
assert np.shape(hpq)==(7, 7)             # H2O has 7 orbitals when using STO-3G basis.
assert np.shape(vpqrs)==(7, 7, 7, 7)

print(hpq)
# print(vpqrs)  # vpqrs is a large tensor，for brevity, we comment this line by default, interested users can uncomment this line to see it.

[[-3.2911e+01  5.5623e-01  2.8755e-01 -5.1653e-15 -7.4568e-02 -9.4552e-02 -2.8670e-01]
 [ 5.5623e-01 -8.0729e+00 -4.0904e-02  1.5532e-15  1.7890e-01  3.5048e-01  1.3460e+00]
 [ 2.8755e-01 -4.0904e-02 -7.3355e+00  6.8611e-16  4.1911e-01  5.2109e-01 -7.0928e-01]
 [-5.1650e-15  1.5617e-15  6.7472e-16 -7.5108e+00  5.4096e-16  1.7947e-15  4.5448e-16]
 [-7.4568e-02  1.7890e-01  4.1911e-01  5.3561e-16 -5.7849e+00  2.0887e+00 -1.2427e-01]
 [-9.4552e-02  3.5048e-01  5.2109e-01  1.8235e-15  2.0887e+00 -5.0803e+00 -1.3967e-02]
 [-2.8670e-01  1.3460e+00 -7.0928e-01  4.8475e-16 -1.2427e-01 -1.3967e-02 -5.0218e+00]]


Most of the time, we don't need to extract those integrals and assemble the Hamiltonian manually, *qchem* has already ecapsulated this procedure in `Molecule` class, we can get the Hamiltonian of water molecule from it.

In [6]:
# Build `Molecule` for water molecule.
from paddle_quantum.qchem import Molecule

mol = Molecule(
    geometry=h2o_coords,
    basis="sto-3g",
    driver=PySCFDriver()
)


H_of_water = mol.get_molecular_hamiltonian()

INFO:root:
#######################################
SCF Calculation (Classical)
#######################################
INFO:root:Basis: sto-3g


converged SCF energy = -73.9677038774737


INFO:root:SCF energy: -73.96770.


Since quantum computing only allows operation based on Pauli operators $\hat{\sigma}_x$, $\hat{\sigma}_y$, $\hat{\sigma}_z$, we have to transform Hamiltonian in above expression to a form represented by Pauli operators. This can be achieved via [Jordan-Wigner 变换](https://en.wikipedia.org/wiki/Jordan%E2%80%93Wigner_transformation) and we have already integrated it in `get_molecular_hamiltonian` method.

In [7]:
print('There are', H_of_water.n_terms, 'terms in H2O Hamiltonian in total.')
print('The first 10 terms are \n', H_of_water[:10])

There are 2110 terms in H2O Hamiltonian in total.
The first 10 terms are 
 -44.808115297466266 I
12.537584025014317 Z0
12.537584025014313 Z1
1.8115178684479893 Z2
1.8115178684479893 Z3
1.4546840922727915 Z4
1.4546840922727915 Z5
1.4299903414012243 Z6
1.429990341401224 Z7
1.0849294605602529 Z8


Great! Now you know how to build a proper Hamiltonian from a given molecular structure, let's move further and see how to use [variational quantum eigensolver](./VQE_EN.ipynb) (VQE) to determine the ground state of hydrogen molecule.


---
## References

[1] [PySCF: Python-based Simulations of Chemistry Framework](https://pyscf.org/)